In [43]:
import geopandas as gpd
import pandas as pd
import os
from pathlib import Path
import warnings
warnings.filterwarnings(action='ignore')

place = 'tel_aviv'
crs_prj = 'EPSG:2039'


# Get the current working directory (e.g., the folder you're running from)
cwd = Path().resolve()

# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)



In [47]:
streets = gpd.read_file(f"{data_folder}/streets.shp")[['oidrechov','highway','geometry']]
# Convert categorical highway values to integer labels
streets['highway'], highway_labels = streets['highway'].factorize()

# bench
bench = gpd.read_file(f"{data_folder}/bench/bench.shp")
streets = streets.merge(bench[['oidrechov','benches_pe']], on='oidrechov').rename(columns={'benches_pe':'bench'})
# green_canopy
green_canopy= gpd.read_file(f"{data_folder}/green_canopy/green_canopy.shp")
streets = streets.merge(green_canopy[['oidrechov','green_perc']], on='oidrechov').rename(columns={'green_perc':'green_canopy'})

# pop_dens
pop_dens= gpd.read_file(f"{data_folder}/pop_dens/pop_dens.shp")
streets = streets.merge(pop_dens[['oidrechov','estimated_']], on='oidrechov').rename(columns={'estimated_':'pop_dens'})
# pop_dens
road_right= gpd.read_file(f"{data_folder}/road_right/road_right.shp")
streets = streets.merge(road_right[['oidrechov','road_right']], on='oidrechov')
streets['road_right'] = streets['road_right'].fillna(-1)

# shadows
shadows= gpd.read_file(f"{data_folder}/shadows/shadows.shp")
streets = streets.merge(shadows[['oidrechov','weighted_s']], on='oidrechov').rename(columns={'weighted_s':'shadows'})

# sidewalk_width
sidewalk_width= gpd.read_file(f"{data_folder}/sidewalk/sidewalk_width.shp")
streets = streets.merge(sidewalk_width[['oidrechov','final']], on='oidrechov').rename(columns={'final':'sidewalk_width'})

# slope
slope = gpd.read_file(f"{data_folder}/slope/slope.shp")
streets = streets.merge(slope[['oidrechov','avg_slope']], on='oidrechov').rename(columns={'avg_slope':'slope'})
streets = streets.reset_index(names='index_str_name')
streets.to_file(f'{data_folder}/streets_1.shp')
streets

,index_str_name,oidrechov,highway,geometry,bench,green_canopy,pop_dens,road_right,shadows,sidewalk_width,slope
0,0,1.0,0,"LINESTRING (184322.705 668574.483, 184351.736 ...",0.0,0.55,0.005694,19.169206,255.000000,3.536674,0.030367
1,1,320.0,0,"LINESTRING (184251.346 668628.811, 184322.705 ...",0.0,0.53,0.005694,18.101117,231.145226,9.824291,0.089168
2,2,9429.0,0,"LINESTRING (184322.705 668574.483, 184297.349 ...",0.0,0.26,0.005694,9.755713,207.291032,4.521383,0.076952
3,3,423.0,0,"LINESTRING (184351.736 668559.201, 184381.018 ...",0.0,0.17,0.005694,10.567812,255.000000,5.684140,0.097721
4,4,9427.0,0,"LINESTRING (184351.736 668559.201, 184371.851 ...",0.0,0.13,0.005694,25.680978,247.066546,7.660572,0.101118
...,...,...,...,...,...,...,...,...,...,...,...
8746,8746,9638.0,10,"LINESTRING (178863.057 665070.191, 178854.901 ...",0.0,0.17,0.024087,19.656900,255.000000,0.000000,0.032017
8747,8747,9643.0,1,"LINESTRING (178841.446 665011.389, 178853.393 ...",0.0,0.29,0.024087,19.785467,127.560380,0.000000,0.157522
8748,8748,9639.0,1,"LINESTRING (178810.907 665023.311, 178806.881 ...",0.0,0.38,0.022666,19.915410,255.000000,0.000000,0.078291
8749,8749,9642.0,1,"LINESTRING (178841.446 665011.389, 178835.182 ...",0.0,0.25,0.024087,20.263228,255.000000,0.870168,0.129780


In [48]:
data_for_model = pd.read_csv(f"{data_folder}/model_data_final.csv").rename(columns={'Unnamed: 0':'index_str_name'})
data_with_new_features = data_for_model.merge(streets, on='index_str_name')
data_with_new_features

,index_str_name,day part,season,day,buildings,businesses,educationa,Health_ser,Leisure_am,Playground,...,oidrechov,highway,geometry_y,bench,green_canopy,pop_dens,road_right,shadows,sidewalk_width,slope
0,0,0,2,0,16.036667,0.0,8,1,0,7,...,1.0,0,"LINESTRING (184322.705 668574.483, 184351.736 ...",0.0,0.55,0.005694,19.169206,255.0,3.536674,0.030367
1,0,1,2,0,16.036667,0.0,8,1,0,7,...,1.0,0,"LINESTRING (184322.705 668574.483, 184351.736 ...",0.0,0.55,0.005694,19.169206,255.0,3.536674,0.030367
2,0,2,2,0,16.036667,0.0,8,1,0,7,...,1.0,0,"LINESTRING (184322.705 668574.483, 184351.736 ...",0.0,0.55,0.005694,19.169206,255.0,3.536674,0.030367
3,0,3,2,0,16.036667,0.0,8,1,0,7,...,1.0,0,"LINESTRING (184322.705 668574.483, 184351.736 ...",0.0,0.55,0.005694,19.169206,255.0,3.536674,0.030367
4,0,0,2,1,16.036667,0.0,8,1,0,7,...,1.0,0,"LINESTRING (184322.705 668574.483, 184351.736 ...",0.0,0.55,0.005694,19.169206,255.0,3.536674,0.030367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473795,8750,3,0,0,19.660000,0.0,9,3,4,2,...,9640.0,1,"LINESTRING (178804.172 665057.617, 178807.785 ...",0.0,0.22,0.022610,19.379924,255.0,0.000000,0.064907
473796,8750,0,0,1,19.660000,0.0,9,3,4,2,...,9640.0,1,"LINESTRING (178804.172 665057.617, 178807.785 ...",0.0,0.22,0.022610,19.379924,255.0,0.000000,0.064907
473797,8750,1,0,1,19.660000,0.0,9,3,4,2,...,9640.0,1,"LINESTRING (178804.172 665057.617, 178807.785 ...",0.0,0.22,0.022610,19.379924,255.0,0.000000,0.064907
473798,8750,2,0,1,19.660000,0.0,9,3,4,2,...,9640.0,1,"LINESTRING (178804.172 665057.617, 178807.785 ...",0.0,0.22,0.022610,19.379924,255.0,0.000000,0.064907


In [51]:

data_with_new_features.drop(columns= ['geometry_y','oidrechov','geometry_x','node_end','node_start','density']).to_csv(f"{data_folder}/model_data_final_1.csv")